## Hello!

About the Take Home Assignment:
The aim of the Take Home Assisnment is to check your understanding of AI-ML concepts, and consists of two simple questions. 
For each question you are expected to do the following:
1) Preprocess the data.
2) Analyse the data (Explain your understanding / observations)
3) Create a model and justify the results.


# Logic
**Q1**
---
I have worked with classifiers like naive bayes earlier as well. I knew that by encoding each unique word by an integer(here I chose it's frequency since now I could distinguish between sentences differing in only word frequency like "good" and "good good" ) I could classify each summary record with a unique feature set and a label(Score)

Now to improve predictions I put score above 3 to be positive else negative
So I had 2 classes and numerical input features. After this I used the Multinomial Naive Bayes(multinomial since input features are non binary) to classify the record
Before running on the model I cleaned/processed the data( removed the Nan values and reduced the dimensions by feature selection)

---
**Q2**
---
I first observed the data and discovered the different classes of flowers and counted them to be 5 and encoded them in integer values
Later I build a pandas data frame with the images (decoded using tensor flow libraries) and their corresponding class(flower encoding)
I randomly took records  from this dataframe and split them in training testing set
Then I build a keras NN model. I used CNN to capture various subtle features of each image( color, shapes etc) 
The first layer was rescaling each color value to range of 0-1 to save computation
Second layer was a convolutional followed by activation layer with relu function ( this improved non linearity of the model). 
I used 3 convo layers followed by a max pooling to decrease the parameters. Then I flattened all the feature maps after convolution, passed them through a hidden layer and condensed then to final 5 classes.

I first got accuracy of 30 %, this was when my training data was small ( about 800) and only 1 convo layer. I gradually increased dataset and added some layers to its final output 

I avoided data augmentation since training and testing the given data was only taking more time.




## Question 1:


Download the Amazon Fine Food Reviews Dataset (https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews?select=Reviews.csv) and build a model to predict the sentiment of the review.

You must start with basic EDA and pre-processing (Explain each step of the model building process).

In [ ]:
import numpy as np
import pandas as pd
import csv

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Reviews.csv',engine="python",on_bad_lines='skip')
# we drop the duplicate values since they are already analysed before
df.drop_duplicates(subset = "Summary",inplace=True)
df.drop_duplicates(subset = "Id",inplace=True)

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295743 entries, 0 to 568452
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      295743 non-null  int64 
 1   ProductId               295743 non-null  object
 2   UserId                  295743 non-null  object
 3   ProfileName             295737 non-null  object
 4   HelpfulnessNumerator    295743 non-null  int64 
 5   HelpfulnessDenominator  295743 non-null  int64 
 6   Score                   295743 non-null  int64 
 7   Time                    295743 non-null  int64 
 8   Summary                 295742 non-null  object
 9   Text                    295743 non-null  object
dtypes: int64(5), object(5)
memory usage: 24.8+ MB
None


In [ ]:
# We dont need all columns(features) for sentinent analysis so we do dimensionality reduction by feature selection
# we drop the rows that containe na values and get the CLEANED Data
# We do sentinent analysis on the Summary of the review(it is already clean and has visual link with the score)
df.drop(df.columns[[0,1,3,4,5,7,9]],axis=1,inplace=True)
df.dropna(subset = "Summary",inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295742 entries, 0 to 568452
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   UserId   295742 non-null  object
 1   Score    295742 non-null  int64 
 2   Summary  295742 non-null  object
dtypes: int64(1), object(2)
memory usage: 9.0+ MB
None


In [ ]:
import re
import pprint

# Function to clean the text by removing the words/characters not used in sentiment analysis
def clean_text(text_array):
    cleaned_array = []
    for text in text_array:
        # Remove non-ASCII characters
        cleaned_text = re.sub(r'[^\x00-\x7F]+', '', text)
        # Remove non-alphanumeric characters
        cleaned_text = re.sub(r'[^a-zA-Z0-9]+', ' ', cleaned_text).replace('.','')
        # Convert to lowercase
        cleaned_text = cleaned_text.lower()
        cleaned_array.append(cleaned_text)
    return cleaned_array

df['Summary'] = clean_text(df['Summary'])
print("the text to analyse has been cleaned")


the text to analyse has been cleaned


In [ ]:
# Well use bag of words Vectorization to build a Numerical Labelled Feature set after which we use Multinomial Naive Bayes to make sentinent prediction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


vectorizer = CountVectorizer()
bag = vectorizer.fit_transform(df['Summary'])

bags = bag.toarray()
print("here the feature set of a record will contain the frequency of each word in the text")

here the feature set of a record will contain the frequency of each word in the text


In [ ]:
# We take random 10000 records, train the sampled dataset and predict on the model
index = np.random.choice(bags.shape[0], size=10000, replace=False)
# arr = [i for i in index]
# print(arr)
arr = [bags[i] for i in index]

In [ ]:
# we build 2 classes: above 3 rating as positive sentiment and below 4 as negative sentinent
df.loc[df["Score"] <= 3, "Score"] = 0
df.loc[df["Score"] > 3 , "Score"] = 1


In [ ]:
# Split the dataset into 80% train and 20% test data
X_train, X_test, Y_train, Y_test = train_test_split(arr, df['Score'][:10000], test_size=0.80, random_state=90)

from sklearn.naive_bayes import MultinomialNB

# We use a multinomial Naive Bayes Classifier
clf = MultinomialNB(alpha=1)

# Train the model
clf.fit(X_train,Y_train)

# clf.predict(X_test)

# Gives the accuracy
print(clf.score(X_test, Y_test))

0.758


## Question 2:


Download the Flower Classification Dataset ("https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz") and build a model to predict the flower.
You should also show the following: 1) Display 4 samples of each type of flower 2) Convert images to grayscale and test the model. 3) Resize the images.

You must start with basic EDA and pre-processing (Explain each step of the model building process).

In [ ]:
## Helper Code to get you started:

import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers,losses
from tensorflow.keras.models import Sequential

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)
print(list(data_dir.glob('*')))


[PosixPath('/root/.keras/datasets/flower_photos/tulips'), PosixPath('/root/.keras/datasets/flower_photos/LICENSE.txt'), PosixPath('/root/.keras/datasets/flower_photos/sunflowers'), PosixPath('/root/.keras/datasets/flower_photos/daisy'), PosixPath('/root/.keras/datasets/flower_photos/roses'), PosixPath('/root/.keras/datasets/flower_photos/dandelion')]


In [ ]:
from sklearn.preprocessing import LabelEncoder
print("Thus there are 5 classes of flowers => ['roses','dandelion','daisy','sunflowers','tulips']")
classes = ['roses','tulips','dandelion','daisy','sunflowers']
le = LabelEncoder()
le.fit(classes)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
# We encode the different labels for a nn can only work with encoded classes
print(le_name_mapping)

Thus there are 5 classes of flowers => ['roses','dandelion','daisy','sunflowers','tulips']
{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [ ]:
All_images = list(data_dir.glob('*/*.jpg'))
print(len(All_images))

3670


In [ ]:
# prepare the dataset, it contains the img matrix( tensored ) and is labeled 
import pandas as pd

def readImage(imagePath):
  # read the image and re scale it to range 0-1 to avoid computation cost
  img = tf.io.read_file(str(imagePath))
  img = tf.image.decode_jpeg(img)
  img = tf.image.resize(img, size = (256,256))
  # img = np.array(plt.imread(str(imagePath))) / 255.0
  return img

# The data frame is randomized and a subset of it is taken as the whole set is computationally extensive
temp = {'image':[],'Class':[]}
# for img in np.random.choice(len(All_images),size=360, replace=False):
#   temp['image'].append(readImage(All_images[img]))
#   temp['Class'].append(le_name_mapping[str(All_images[img]).split('/')[-2]])
np.random.shuffle(All_images)
for img in All_images:
  temp['image'].append(readImage(img))
  temp['Class'].append(le_name_mapping[str(img).split('/')[-2]])

df = pd.DataFrame(temp)
x = np.array(df['image'])

In [ ]:
# prepare the labels test and training set
labels = tf.convert_to_tensor(df['Class'][:-200])
tlabels = tf.convert_to_tensor(df['Class'][-200:-1])

In [ ]:
# prepare the features train and test set
features = tf.stack(x[:-200])
tfeatures = tf.stack(x[-200:-1])

In [ ]:
# make the model (CNN). We add only 1 Convolutionary layer
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(256,256,3)),
    tf.keras.layers.Rescaling(1/255.0), # this layes re schales the input to range of 0-1 for saving computation further
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax') # softmax because there are multiple classes
])

In [ ]:
model.compile(optimizer='adam',loss=losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       7

In [ ]:
# By Adding more layers the accuracy increased by about 8%
# Also did pooling because the parameters were becoming way too large
print("Fit model on training data")
history = model.fit(
    features,
    labels,
    batch_size=64,
    epochs=3,
)

Fit model on training data
Epoch 1/3
55/55 [==============================] - 485s 9s/step - loss: 1.4279 - accuracy: 0.4207
Epoch 2/3
55/55 [==============================] - 471s 9s/step - loss: 1.0174 - accuracy: 0.6089
Epoch 3/3
55/55 [==============================] - 475s 9s/step - loss: 0.8171 - accuracy: 0.6919


In [ ]:
results = model.evaluate(tfeatures, tlabels, batch_size=64)

4/4 [==============================] - 8s 2s/step - loss: 0.9030 - accuracy: 0.6482


In [ ]:
print("After increasing the dataset size from 1000 -> 2000, the accuracy increased by 15%")
print("I didnt do data augmentation because i wasnt short of training data")

After increasing the dataset size from 1000 -> 2000, the accuracy increased by 15%
I didnt do data augmentation because i wasnt short of training data
